官方文档：https://github.com/openatx/facebook-wda

小技巧：
利用 element.bounds 可以返回元素位置

In [3]:
import wda
import time
import json
import threading
wda.DEBUG = False # default False
wda.HTTP_TIMEOUT = 60.0 # default 60.0 seconds

用这个网站找bundle id:https://offcornerdev.com/bundleid.html

In [4]:
# get env from $DEVICE_URL if no arguments pass to wda.Client
# http://localhost:8100 is the default value if $DEVICE_URL is empty
c = wda.Client('http://10.0.0.96:8100')
# 测试连接是否成功
print(c.status())

# s = c.session('com.gemd.iting')
s = c.session('com.apple.Preferences')
s.tap(0, 0)

{'state': 'success', 'os': {'name': 'iOS', 'version': '12.1.4'}, 'ios': {'simulatorVersion': '12.1.4', 'ip': '10.0.0.96'}, 'build': {'time': 'Mar 17 2019 19:58:04'}, 'sessionId': '62E87789-028A-4113-BC43-A5C37D98E34C'}


GenericDict(value={}, sessionId='B0194E4B-5D2C-4585-AD80-C85EBA020A5C', status=0)

In [90]:
c.source() #xml
# dic_data = c.source(accessible=True) # default false, format JSON

'<?xml version="1.0" encoding="UTF-8"?>\n<XCUIElementTypeApplication type="XCUIElementTypeApplication" name="喜马拉雅" label="喜马拉雅" enabled="true" visible="true" x="0" y="0" width="1024" height="1366">\n  <XCUIElementTypeOther type="XCUIElementTypeOther" enabled="true" visible="false" x="0" y="0" width="1024" height="1366"/>\n  <XCUIElementTypeWindow type="XCUIElementTypeWindow" enabled="true" visible="true" x="0" y="0" width="1024" height="1366">\n    <XCUIElementTypeOther type="XCUIElementTypeOther" enabled="true" visible="true" x="0" y="0" width="1024" height="1366">\n      <XCUIElementTypeOther type="XCUIElementTypeOther" enabled="true" visible="true" x="0" y="0" width="1024" height="1366">\n        <XCUIElementTypeImage type="XCUIElementTypeImage" name="leftside_shadow_bg" enabled="true" visible="false" x="-14" y="0" width="14" height="1366"/>\n        <XCUIElementTypeOther type="XCUIElementTypeOther" enabled="true" visible="true" x="0" y="0" width="1024" height="1366">\n          <XC

TypeError: 'module' object is not callable

In [5]:
text_box = s(name='说点什么...') #定义输入栏
send_text_button = s(className='Button',label="发送") #定义发送按钮
minimize_button = s(className='Button', name='最小化') #定义最小化按钮
leave_button = s(name='live audience btn close') #定义离开按钮
global_play_button = s(name="全局播放器") #定义全局播放器按钮

position_dic = {'退出':(980,60), '最小化':(520,800), '全局播放器':(500,1330),'发送文本':(950, 1200), '@ta':(550,1330)}

In [6]:
# s.click(*position_dic["最小化"])

In [7]:
# 版本一：返回名字或None （核心程序）
def get_name():
    name_str = s(nameContains=' 进入直播间')
    #     print(name_str.exists) #打印是否存在
    if name_str.exists: #用exists 检查是否存在，这个速度很快

        raw_str = name_str.text #提取text，速度慢
        if raw_str:
            name = raw_str[:-6]
            return name
        return None


In [8]:
# 版本二：包含try的版本，返回名字或None （核心程序）
def get_name_with_try():
    name_str = s(nameContains=' 进入直播间')
    #     print(name_str.exists) #打印是否存在
    if name_str.exists: #用exists 检查是否存在，这个速度很快

        try: #这里有时候会出问题，所以用个try
            raw_str = name_str.text #提取text，速度慢
            if raw_str:
                name = raw_str[:-6]
                return name
            return None
        except:
            pass

In [9]:
# 退出房间再进入房间（用来减少缓存）
def out_and_reenter_room():
    s.tap(*position_dic['退出'])
    s.tap(*position_dic['最小化'])
    s.tap(*position_dic['全局播放器'])


In [10]:
# 发送信息
def send_text(text_content):
    text_box.set_text(text_content) 
    s.tap(*position_dic['发送文本']) #按下发送按钮
    s.tap(*position_dic['最小化'])

In [11]:
# 循环回复别人的@

def continue_repy_at():
    while True:
        at_name = '凡鲁'
        element_list = s(nameContains='@'+at_name).find_elements()
        for e in element_list:
            if e.displayed:
                print(e.text)
                rect = e.bounds
                s.tap(rect.x/2,rect.y-25) 
                s.tap(*position_dic['@ta']) #点开@ ta
                reply_thread = threading.Thread(target = send_text, arg=input('回复：'),)
                reply_thread.start()
                reply_thread.join()

In [12]:
# 打印别人的@
def show_at():
    while True:
        at_name = '凡鲁'
        element_list = s(nameContains='@'+at_name).find_elements()
        for e in element_list:
            if e.displayed:
                print(e.text)
                rect = e.bounds
                s.tap(rect.x/2,rect.y-25) 
                s.tap(*position_dic['@ta']) #点开@ ta
                reply_thread = threading.Thread(target = send_text, arg=input('回复：'),)
                reply_thread.start()
                reply_thread.join()

In [13]:
# 发送我输入的文字
def continue_input_text():
    while True:
        send_text(input('输入要说的话：'))
        print('发送成功')

In [14]:
# send_input_text()

In [15]:
# 禁止名单,遇到这些人，会直接continue循环,不会对这些人打招呼
banned_list = ['凡鲁']

In [28]:
# ------主程序------




last_time = time.time()
send_time = 0

while True:
    name = get_name_with_try() #获取进入房间的人的名字
    if name:
        send_time_begin = time.time()
        print(name)
        if name in banned_list:
            continue
        text_content = '欢迎～～'+ name #在这里输入你要发送的
        send_text(text_content)
        send_time = time.time()-send_time_begin #这里计算发送信息花的时间
        print(send_time,'\n')
        time.sleep(10)



#     break #如果只想发送一次，就打开这个break
# 计算使用的时间，如果时间过长，出房间再进房间
    use_time = time.time()-last_time - send_time #除去发送，剩下花的时间（循环一次的时间）

    if use_time > 3:
        print(use_time,'\n')
        out_and_reenter_room()

    last_time = time.time()
    send_time = 0

1868595gejf
10.734290838241577 

12.375375032424927 

盖光层层
5.1002678871154785 

10.707144260406494 

泡jiojio女孩


KeyboardInterrupt: 